In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import math
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.optim as optim


#### Transformer Encoder

In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    

class Transformer(nn.Module):
    def __init__(self, d_model, heads, num_layers,dropout, max_len, output_classes):
        super().__init__()
        self.encoder_layer= nn.TransformerEncoderLayer(d_model=d_model, nhead=heads)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.positional_encoder = PositionalEncoding(d_model=d_model, dropout=dropout, max_len=max_len)
        self.linear = nn.Linear(in_features=d_model, out_features=output_classes)
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self, x):
        x1 = self.positional_encoder(x)
        x2 = self.encoder(x1)
        out = self.linear(x2)
        out = self.softmax(out)
        return out
        


kinematic_features = 22 #dmodel
heads = 11
num_layers = 6
dropout = 0.1
max_len = 100
output_classes = 7

# src = torch.rand(10, 10, kinematic_features)

model = Transformer(kinematic_features,heads,num_layers,dropout,max_len,output_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)


Transformer(
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=22, out_features=22, bias=True)
    )
    (linear1): Linear(in_features=22, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=22, bias=True)
    (norm1): LayerNorm((22,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((22,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=22, out_features=22, bias=True)
        )
        (linear1): Linear(in_features=22, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(

#### Load Data

In [4]:
gesture_labels_directory = './new_dataset/peg_transfer/peg_transfer/labeled/gestures/'
window_size = 10 # 10 frame windows (kinematic data)
enite_dataset = []
counter = 0



class KinematicDataset(Dataset):
    
    def __init__(self,data_directory, window_size):
        
        self.directory = data_directory
        self.window_size = window_size
        self.le = preprocessing.LabelEncoder()
        self.enc = preprocessing.OneHotEncoder()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)
        (self.X, self.Y) = self._load_data()
        self.X = self.X.to(self.device)
        self.Y = self.Y.to(self.device)
        
        
    def _load_data(self):
        x = []
        y = []
        self.windows_per_file = []
        
        for filename in os.listdir(self.directory):
            f = os.path.join(self.directory, filename)
            # checking if it is a file
            if os.path.isfile(f):
                kinematics_data = pd.read_csv(f)
                kin_data = kinematics_data.iloc[:,:-1]
                kin_label = kinematics_data.iloc[:,-1]
                
                x.append(kin_data.values)
                y.append(kin_label.values)
                
                
                
                if len(self.windows_per_file) == 0:
                    self.windows_per_file = [len(kin_label) - self.window_size + 1]
                else:
                    self.windows_per_file.append(self.windows_per_file[-1] + len(kin_label) - self.window_size + 1)
        
        self.le.fit(y[0])
        y = [self.le.transform(yi) for yi in y]
        x = np.concatenate(x)
        y = np.concatenate(y)
        
        x = torch.from_numpy(x)
        y = torch.from_numpy(y)
        
        x = x.to(torch.float32)
        return (x, y)
    
    def __len__(self):
        # this should return the size of the dataset
        return self.Y.shape[0]-window_size
    
    def __getitem__(self, idx):
        # this should return one sample from the dataset
        features = self.X[idx : idx + window_size]
        target = self.Y[idx : idx + window_size]
        return features, target
                    
                    


In [5]:
# train_data, test_data = 

batch_size = 32

dataset = KinematicDataset(gesture_labels_directory,10)

# use 20% of training data for validation
train_set_size = int(len(dataset) * 0.8)
valid_set_size = len(dataset) - train_set_size
print(train_set_size, valid_set_size)

seed = torch.Generator().manual_seed(42)
train_set, test_set = torch.utils.data.random_split(dataset, [train_set_size, valid_set_size],generator=seed)


trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

print(train_set.dataset.X.shape, test_set.dataset.X.shape)
print(train_set.dataset.Y.shape, test_set.dataset.Y.shape)


cuda
119279 29820
torch.Size([149109, 22]) torch.Size([149109, 22])
torch.Size([149109]) torch.Size([149109])


In [6]:
# loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
epochs = 1

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        
        print(inputs[0,:,:].shape)
        print(outputs[0,:,:].shape)
        print(labels.shape)
        break
        outputs = torch.transpose(outputs,1,2)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

torch.Size([10, 22])
torch.Size([10, 7])
torch.Size([32, 10])
Finished Training


In [8]:
MODEL_NAME = "MODEL_V0.pth"
MODEL_SAVE_PATH = "./models/"+MODEL_NAME

torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)

In [9]:
#Test

# y_pred = model(list(iter(test_set)))

for i,data in enumerate(testloader):
    x,y = data
    
    y_pred = model(x)
    
    y_pred = y_pred.cpu()
    y_pred = y_pred.detach().numpy()
    
    print(np.argmax(y_pred[0],axis=1),y[0])

[4 1 5 4 1 3 1 1 6 2] tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0')
[1 6 3 6 4 4 0 3 1 0] tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
[4 2 6 1 1 3 3 5 6 2] tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
[3 1 1 5 4 4 2 3 5 0] tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
[2 5 5 1 4 4 6 3 1 0] tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4], device='cuda:0')
[3 2 4 3 1 4 5 3 0 5] tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
[1 5 1 2 4 2 3 3 5 4] tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4], device='cuda:0')
[1 3 3 4 2 6 0 6 1 4] tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
[3 4 6 2 0 3 6 1 2 1] tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
[4 2 0 4 5 1 3 5 3 5] tensor([5, 5, 5, 6, 6, 6, 6, 6, 6, 6], device='cuda:0')
[0 1 3 5 5 6 5 1 4 4] tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
[0 4 3 1 1 4 3 6 2 0] tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
[2 5 1 0 5 3 6 1 0 3] tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dev